In [1]:
#
# cs590-Module7-mongo.ipynb - This is a Jupyter Notebook designed to 
# connect to a MongoDB database and provide CRUD operations.
#
# This notebook uses a supplemental configuration file that has the
# following format:
# 
# [mongodb]
# host=localhost
# database=delisweet
# collection=REPLACE_WITH_YOUR_COLLECTION_AS_NEEDED
# user=''
# password=REPLACE_WITH_YOUR_PASSWORD_FOR_THE_DATABASE_USER
# port=27017
#
# We also build a config function to read the data from the
# configuration file to make the code easier to maintain and more
# secure. 
#
# Remember: You should _NEVER_ embed a password in your 
# source code!
#
#------------------------------------------------------------------
# Change History
#------------------------------------------------------------------
# Version   |   Description
#------------------------------------------------------------------
#    1          Initial Development
#------------------------------------------------------------------

import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
from configparser import ConfigParser

#
# db - variable to identify the database we are communicating with
#
db = 'delisweet'

#
# col - variable identify the collection we are working with
#
col = ''

def config(filename, section):
    cp = ConfigParser()
    cp.read(filename)
    block = {}
    if cp.has_section(section):
        items = cp.items(section)
        for item in items:
            block[item[0]] = item[1]
    else:
        raise Exception(f"Section {section} was not found in the {filename} file!")
    return block

def displayConfig(filename, section):
    confData = config(filename, section)
    print(confData)

def dbConnect(filename, section):
    confData = config(filename, section)

    # Ensure our global db/col match the configuration
    global db, col
    db = confData.get("database", db)
    col = confData.get("collection", col)

    conn = None
    try:
        host = confData["host"]
        port = int(confData["port"])
        user = confData.get("user", "").strip()
        password = confData.get("password", "").strip()

        # Build URI (works with or without credentials)
        if user and password:
            uri = f"mongodb://{user}:{password}@{host}:{port}"
        else:
            uri = f"mongodb://{host}:{port}"

        conn = MongoClient(uri)

        # Touch the DB/collection to validate they exist
        _database = conn[db]
        _collection = _database[col]
        _collection.estimated_document_count()

        return conn
    except (Exception, pymongo.errors.ConnectionFailure) as error:
        print(error)
        return None

def dbTest(filename, section):
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    database = conn[db]
    collection = database[col]
    try:
        numDocs = collection.count_documents({})
        print(f'There are {numDocs} in the {col} collection')
    except (Exception, pymongo.errors.ConnectionFailure) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def createRecord(data, conn):
    try:
        database = conn[db]
        collection = database[col]
        result = collection.insert_many(data)
        return result.inserted_ids
    except Exception as error:
        print(error)
        return None

def testCreateRecord(filename, section):
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    data = [
        {"row": {
            "Nutrient code": "777",
            "Nutrient description": "Zinc",
            "Nutrient description abbrev": "Zn",
            "Nutrient unit": "mcg",
            "Date added": "05/08/2025",
            "Last modified": "05/08/2025"
        }}
    ]
    createRecord(data, conn)
    print('Record Added')
    conn.close()

def readRecord(clause, conn):
    try:
        database = conn[db]
        collection = database[col]
        return list(collection.find(clause))
    except Exception as error:
        print(error)
        return None

def testReadRecord(filename, section):
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    clause = {"row.Nutrient code": "777"}
    records = readRecord(clause, conn)
    print('Records Retrieved:')
    print(records)
    conn.close()

def updateRecord(select, clause, conn):
    try:
        database = conn[db]
        collection = database[col]
        result = collection.update_many(select, clause)
        return result.modified_count
    except Exception as error:
        print(error)
        return 0

def testUpdateRecord(filename, section):
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    select = {"row.Nutrient code": "777"}
    clause = {"$set": {"row.Nutrient description": "ZINC"}}
    res = updateRecord(select, clause, conn)
    print(f'{res} record(s) have been updated.')
    conn.close()

def deleteRecord(select, conn):
    try:
        database = conn[db]
        collection = database[col]
        result = collection.delete_many(select)
        return result.deleted_count
    except Exception as error:
        print(error)
        return 0

def testDeleteRecord(filename, section):
    print('Connecting to our MongoDB database...')
    conn = dbConnect(filename, section)
    select = {"row.Nutrient code": "777"}
    ret = deleteRecord(select, conn)
    print(f'{ret} record(s) were removed from the collection.')
    conn.close()

if __name__ == '__main__':
    displayConfig('database.conf','mongodb')
    dbTest('database.conf','mongodb')
    testCreateRecord('database.conf','mongodb')
    testReadRecord('database.conf','mongodb')
    testUpdateRecord('database.conf','mongodb')
    testDeleteRecord('database.conf','mongodb')


{'host': 'localhost', 'database': 'delisweet', 'collection': 'integration_test', 'user': '', 'password': '', 'port': '27017'}
Connecting to our MongoDB database...
There are 0 in the integration_test collection
Connecting to our MongoDB database...
Record Added
Connecting to our MongoDB database...
Records Retrieved:
[{'_id': ObjectId('695afc88345dd5a07848cbec'), 'row': {'Nutrient code': '777', 'Nutrient description': 'Zinc', 'Nutrient description abbrev': 'Zn', 'Nutrient unit': 'mcg', 'Date added': '05/08/2025', 'Last modified': '05/08/2025'}}]
Connecting to our MongoDB database...
1 record(s) have been updated.
Connecting to our MongoDB database...
1 record(s) were removed from the collection.
